In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# -------------------------------------------------
# 1) Load final cleaned dataset
# -------------------------------------------------
df = pd.read_csv("NGS2020_selected_renamed.csv")

# -------------------------------------------------
# 2) Prepare ln(earnings)
# -------------------------------------------------
df["JobIncomeCategory"] = pd.to_numeric(df["JobIncomeCategory"], errors="coerce")

# Keep valid positive income only
df = df.dropna(subset=["JobIncomeCategory"]).copy()
df = df[df["JobIncomeCategory"] > 0].copy()

df["LnEarnings"] = np.log(df["JobIncomeCategory"])

# -------------------------------------------------
# 3) Main variables (funding dummies)
# Baseline omitted: OtherFunding
# -------------------------------------------------
funding_vars = [
    "GovLoan", "RESP", "GovGrant", "NonGovGrant", "Scholarship",
    "Savings", "FamilySupport", "BankLoan", "CreditCard", "EmployerFunding"
]
funding_vars = [v for v in funding_vars if v in df.columns]

# -------------------------------------------------
# 4) Control variables (already dummied)
# WorkPlacement removed
# -------------------------------------------------
control_vars = [
    "Male",
    "Ontario", "Quebec", "WesternProv",
    "ageUnder25", "age25to29", "age30to39", "age40more",
    "status_ctz_birth", "status_ctz_nature", "status_immigrant",
    "edu_college", "edu_bachelor", "edu_graduate",
    "fieldOfStudy_edu", "fieldOfStudy_human", "fieldOfStudy_social",
    "fieldOfStudy_business", "fieldOfStudy_lifeScience",
    "fieldOfStudy_math", "fieldOfStudy_engineering", "fieldOfStudy_health",
    "WorkBeforeProgram",
    "Parent1Edu_higher", "Parent1Edu_same", "Parent1Edu_lower",
    "Parent2Edu_higher", "Parent2Edu_same", "Parent2Edu_lower"
]
control_vars = [v for v in control_vars if v in df.columns]

# -------------------------------------------------
# 5) Build X and y, then drop any missing rows
# -------------------------------------------------
X = df[funding_vars + control_vars].copy()
y = df["LnEarnings"]

reg_data = pd.concat([y, X], axis=1).dropna().copy()
y_clean = reg_data["LnEarnings"]
X_clean = sm.add_constant(reg_data[funding_vars + control_vars])

# -------------------------------------------------
# 6) OLS with standard SEs
# -------------------------------------------------
model_ols = sm.OLS(y_clean, X_clean).fit()

# -------------------------------------------------
# 7) Robustness check: Robust SEs (HC1)
# -------------------------------------------------
model_robust = sm.OLS(y_clean, X_clean).fit(cov_type="HC1")

# -------------------------------------------------
# 8) Print results
# -------------------------------------------------
print("\n========= OLS (Standard SEs) =========")
print(model_ols.summary())

print("\n========= OLS (Robust SEs: HC1) =========")
print(model_robust.summary())

# -------------------------------------------------
# 9) Side-by-side comparison table
# -------------------------------------------------
compare = pd.DataFrame({
    "coef_OLS": model_ols.params,
    "se_OLS": model_ols.bse,
    "t_OLS": model_ols.tvalues,
    "p_OLS": model_ols.pvalues,
    "coef_Robust": model_robust.params,
    "se_Robust": model_robust.bse,
    "t_Robust": model_robust.tvalues,
    "p_Robust": model_robust.pvalues
})

print("\n========= Coefficient Comparison (OLS vs Robust) =========")
print(compare)

print("\nN used in regression:", int(model_ols.nobs))



========= OLS (Standard SEs) =========
                            OLS Regression Results                            
Dep. Variable:             LnEarnings   R-squared:                       0.212
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     85.55
Date:                Sat, 22 Nov 2025   Prob (F-statistic):               0.00
Time:                        23:00:32   Log-Likelihood:                -9488.8
No. Observations:               12441   AIC:                         1.906e+04
Df Residuals:                   12401   BIC:                         1.935e+04
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------